In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import itertools as it

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.utils import shuffle

In [ ]:
###Base Model
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

# Code for model training
# Training Data

In [ ]:
training_dataa=[]
training_labell=[]

window=30
stride=int(window/2)  

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)

    for m in range(len(Files)):
        if m not in (3,7,13,17,23,27,33,37,43,47,53,57,63,67,73,77,83,87,93,97):
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp = df[i].split()
                p.append(temp)
            p = np.array(p, 'float32')

            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))

                training_dataa.append(new_mat)
                training_labell.append(k)

training_data = training_dataa
training_label = training_labell

print(len(training_dataa))
print(len(training_labell))

In [ ]:
#For DISTANCE features
training_data_dis = np.array(training_data,'float64')
training_data_dis = training_data_dis.reshape(len(training_data_dis),row,column,1)

training_label=np.array(training_label)
print(training_data_dis.shape)
print(training_label.shape)
print(type(training_data_dis))
print(type(training_label))

# Testing Data

In [ ]:
testing_dataa=[]
testing_labell=[]

window = 30
stride = int(window/2)

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)
         
    for m in range(len(Files)):
        if m in (5,6,7,8,9,15,16,17,18,19):
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp=df[i].split()
                p.append(temp)
            p = np.array(p,'float32')
            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat)) 
                testing_dataa.append(new_mat)
                testing_labell.append(k)
testing_data=testing_dataa          
testing_label=testing_labell           

print(len(testing_data))
print(len(testing_label))

In [ ]:
row = len(testing_data[0])
column = len(testing_data[0][0])
print('row: ', row)
print('column: ', column)

In [ ]:
#Distance Feature
testing_data_dis = np.array(testing_data, 'float32')
testing_data_dis = testing_data_dis.reshape((len(testing_data_dis),row,column,1))

testing_label=np.array(testing_label)
print(testing_data_dis.shape)
print(testing_label.shape)

In [ ]:
#converting the data labels to categorical
training_label = keras.utils.to_categorical(training_label, 7)
testing_label= keras.utils.to_categorical(testing_label, 7)

# print(training_label.shape)
print(testing_label.shape)

In [ ]:
#The best model with 3D skeleton data 
row = row
column = column

kinect_input = Input(shape=(row, column, 1))

x = Conv2D(64, (3,3), activation='relu',padding='same',kernel_initializer='he_normal', \
           kernel_regularizer=l2(0.001))(kinect_input)
x = Dropout(0.5)(x)
x = Conv2D(64, (3,3), activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.5)(x)
x = Conv2D(128, (3,3), activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
x = Conv2D(128, (3,3), activation='relu',padding='same',kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.7)(x)
predictions = Dense(7, activation = 'softmax')(x)

base_model = Model(inputs = kinect_input, output = predictions)

adam = keras.optimizers.Adam(lr= 0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

base_model.compile(loss='categorical_crossentropy', optimizer=adam,  metrics=['accuracy'])
base_model.summary()

In [ ]:
checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

history = base_model.fit([training_data_dis], 
                         training_label,
                batch_size=32,
                epochs=100,
                verbose=1,
                validation_data=([testing_data_dis], testing_label),
                callbacks=[checkpoint])

In [ ]:
base_model.load_weights(filepath)
test_pur = base_model.predict([testing_data_dis], batch_size=32)

In [ ]:
match_1 = 0
for i in range(len(testing_label)):
    if np.argmax(testing_label[i]) == np.argmax(test_pur[i]):
        match_1+=1
accuracy_1 = (match_1/len(testing_label))*100
print('left_hand accuracy :', accuracy_1)

In [ ]:
print(np.mean(history.history['acc']))
print(np.mean(history.history['val_acc']))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# (history.history['acc'], history.history['val_acc'])
for i in range(len(history.history['acc'])):
    print(i,history.history['acc'][i],history.history['val_acc'][i])

# plotting the confusion metric

In [ ]:
y_true = []
y_pred = []
for i in range(len(testing_label)):
    y = np.argmax(testing_label[i])
    y_true.append(y)

for i in range(len(test_pur)):
    y = np.argmax(test_pur[i]) #Make changes here
    y_pred.append(y)
print(len(y_true))
print(len(y_pred))

In [ ]:

cl_name = np.array(['Action-0','Action-1','Action-2','Action-3','Action-4','Action-5','Action-6'])
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

#     print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cnf_matrix = confusion_matrix(y_true, y_pred)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name,
                      title='Confusion matrix')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# Code for transfer learning


In [ ]:
import os
os.getcwd()

In [ ]:
training_dataa=[]
training_labell=[]

window=30
stride=int(window/2)

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)
    for m in range(len(Files)):
        if m in (0,1):  #unchanged
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp = df[i].split()
                p.append(temp)
            p = np.array(p, 'float32')
            
            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))
                training_dataa.append(new_mat)
                training_labell.append(k)
   
train_data = training_dataa
train_label = training_labell
print(len(train_data))
print(len(train_label))

In [ ]:
for i in range(7):
    print(i,train_label.count(i))

In [ ]:
train_data_tf = np.array(train_data,'float32')
train_data_tf = train_data_tf.reshape(len(train_data_tf),30,136,1)

train_label_tf=np.array(train_label) 
print(train_data_tf.shape)
print(train_label_tf.shape)

In [ ]:
testing_dataa=[]
testing_labell=[]

window = 30
stride = int(window/2)

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)
    for m in range(len(Files)):
        if m in (5,6,7,8,9,15,16,17,18,19):   
            print(Files[m])
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp = df[i].split()
                p.append(temp)
            p = np.array(p, 'float32')
            
            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))
                testing_dataa.append(new_mat)
                testing_labell.append(k)                          
                
test_data = testing_dataa
test_label = testing_labell

print(len(test_data))
print(len(test_label))

In [ ]:
for i in range(7):
    print(i,test_label.count(i))

In [ ]:
test_data_tf = np.array(test_data, 'float32') 
test_data_tf = test_data_tf.reshape((len(test_data_tf),30,136,1))

test_label_tf=np.array(test_label)
print(test_data_tf.shape)
print(test_label_tf.shape)

In [ ]:
#Converting the categorical labels into one_hot_coding
train_label_tf = keras.utils.to_categorical(train_label_tf, 7)
test_label_tf = keras.utils.to_categorical(test_label_tf, 7)

In [ ]:
print(train_label_tf.shape)
print(test_label_tf.shape)

In [ ]:
filepath= 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\dis.hdf5'
os.path.exists(filepath)

In [ ]:
base_model.load_weights(filepath)
test_pur = base_model.predict([test_data_tf], batch_size=32)

In [ ]:
match_1 = 0
for i in range(len(test_label_tf)):
    if np.argmax(test_label_tf[i]) == np.argmax(test_pur[i]):
        match_1+=1
accuracy_1 = (match_1/len(test_label_tf))*100
print('accuracy :', accuracy_1)
print(test_label_tf.shape)

In [ ]:
filepath_iter_act = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\dis_01.hdf5'
os.path.exists(filepath_iter_act)
#change the filepath

In [ ]:
#Applying Transfer learning, 11 is the  basic
from keras.callbacks import ModelCheckpoint

for layer in base_model.layers[:10]:
        layer.trainable = False
for layer in base_model.layers[10:]:  
        layer.trainable = True
        
new_model = Model(inputs=base_model.input, outputs=predictions)  

adam = keras.optimizers.Adam(lr= 0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model.compile(loss='categorical_crossentropy', optimizer=adam,  metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath_iter_act,
                               monitor='val_acc',
                               verbose=1, 
                               save_best_only=True)

history = base_model.fit(train_data_tf, train_label_tf, 
              batch_size=16, 
              epochs=100, 
              verbose=1, 
              validation_data=(test_data_tf, test_label_tf),
              callbacks=[checkpointer])

In [ ]:
base_model.load_weights(filepath_iter_act)
test_pur = base_model.predict([test_data_tf],batch_size=32)

In [ ]:
match_1 = 0
for i in range(len(test_label_tf)):
    if np.argmax(test_label_tf[i]) == np.argmax(test_pur[i]):
        match_1+=1
accuracy_1 = (match_1/len(test_label_tf))*100
print('accuracy :', accuracy_1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_true = []
y_pred = []
for i in range(len(test_label_tf)):
    y = np.argmax(test_label_tf[i])
    y_true.append(y)

for i in range(len(test_pur)):
    y = np.argmax(test_pur[i]) #Make changes here
    y_pred.append(y)
print(len(y_true))
print(len(y_pred))

In [ ]:

cl_name = np.array(['Action-0','Action-1','Action-2','Action-3','Action-4','Action-5','Action-6'])
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

#     print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cnf_matrix = confusion_matrix(y_true, y_pred)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name,
                      title='Confusion matrix')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
import os
os.getcwd()
os.chdir('C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data')

# Code for Iterative Boosting

In [ ]:
# Testing with 1 or 2 experiments
testing_dataa=[]
testing_labell=[]

window = 30
stride = int(window/2)

for k in range(7):
    Files=[]
    
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)
         
    for m in range(len(Files)):
#         if m in (28,29):
#             print(Files[m])
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp=df[i].split()
                p.append(temp)
            p = np.array(p,'float32')

            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))        
                testing_dataa.append(new_mat)
                testing_labell.append(k)
    testing_data_234=testing_dataa          
    testing_label_234=testing_labell           

print(len(testing_dataa))
print(len(testing_labell))

In [ ]:
for i in range(7):
    print(testing_label_234.count(i))

In [ ]:
row = len(testing_data_234[0])
column = len(testing_data_234[0][0])
print('row: ', row)
print('column: ', column)

In [ ]:
#Distance Feature
testing_data_dis_ts = np.array(testing_data_234, 'float64')
testing_data_dis_ts = testing_data_dis_ts.reshape((len(testing_data_dis_ts),row,column,1))

testing_label_ts=np.array(testing_label_234)
print(testing_data_dis_ts.shape)
print(testing_label_ts.shape)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\dis_all_01.hdf5'
os.path.exists(filepath)

In [ ]:
base_model.load_weights(filepath)
test_pur_234 = base_model.predict([testing_data_dis_ts], batch_size=32)

In [ ]:
#converting the data labels to categorical
# training_label = keras.utils.to_categorical(training_label, 7)
testing_label_ts= keras.utils.to_categorical(testing_label_ts, 7)
print(testing_label_ts.shape)

In [ ]:
match_1 = 0
for i in range(len(testing_label_ts)):
    if np.argmax(testing_label_ts[i]) == np.argmax(test_pur_234[i]):
        match_1+=1
accuracy_1 = (match_1/len(testing_label_ts))*100
print('left_hand accuracy:', accuracy_1)

In [ ]:
y_true = []
y_pred = []
for i in range(len(testing_label_ts)):
    y = np.argmax(testing_label_ts[i])
    y_true.append(y)

for i in range(len(test_pur_234)):
    y = np.argmax(test_pur_234[i]) #Make changes here
    y_pred.append(y)
print(len(y_true))
print(len(y_pred))

In [ ]:
cl_name = np.array(['Action-0','Action-1','Action-2','Action-3','Action-4','Action-5','Action-6'])
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

#     print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cnf_matrix = confusion_matrix(y_true, y_pred)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name,
                      title='Confusion matrix')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# Dealing with misclassified samples

In [ ]:
training_dataa=[]
training_labell=[]

window=30
stride=int(window/2)

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/1.Distance/Testing/subject_1/Action_{}/*.txt'.format(k))):
        Files.append(file)
    for m in range(len(Files)):
        if m in (5,6,7,8,9,15,16,17,18,19): #unchaged
            print(Files[m])
            f = open(Files[m], 'r')
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp = df[i].split()
                p.append(temp)
            p = np.array(p, 'float32')
            
            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))
                training_dataa.append(new_mat)
                training_labell.append(k)       

train_data_miss = training_dataa
train_label_miss = training_labell

print(len(train_data_miss))
print(len(train_label_miss))

In [ ]:
for i in range(7):
    print(i,train_label_miss.count(i))

In [ ]:
print(len(train_data_miss))
print(len(train_label_miss))

In [ ]:
for i in range(len(test_pur_234)): ##identified missalpled from 234
    if np.argmax(testing_label_ts[i]) != np.argmax(test_pur_234[i]):
        x = np.argmax(testing_label_ts[i])
        if x in (0,1,2,3,4,5,6):
            train_data_miss.append(testing_data_234[i])       
            train_label_miss.append(np.argmax(testing_label_ts[i]))

In [ ]:
print(len(train_data_miss))
print(len(train_label_miss))
# print(len(train_data_miss_NN))
# print(len(train_label_miss_NN))

In [ ]:
#Current_BEST MODEL
train_data_mis = np.array(train_data_miss,'float32')
train_data_mis = train_data_mis.reshape(len(train_data_mis),30,136,1)

train_label_mis=np.array(train_label_miss)
print(train_data_mis.shape)
print(train_label_mis.shape)

In [ ]:
#converting the data labels to categorical
train_label_mis = keras.utils.to_categorical(train_label_mis, 7)

print(train_label_mis.shape)

In [ ]:
testing_dataa=[]
testing_labell=[]

window = 30
stride = int(window/2)

for k in range(7):
    Files=[]
    for file in sorted(glob.glob('./ALR_2/Copy_2/Coordinate/Testing/subject_1/Action_{}/*.txt'.format(k))):

        Files.append(file)
    for m in range(len(Files)):    
        if m in (5,6,7,8,9,15,16,17,18,19): #unchaged
            f = open(Files[m], 'r')
#             print(Files[m])
            df = f.readlines()
            f.close()
            p=[]
            for i in range(len(df)):
                temp=df[i].split()
                p.append(temp)
            p = np.array(p,'float32')                

            for i in range(0,len(p)-stride, stride):
                new_mat = p[i:i+window]
                where_are_NaNs = np.isnan(new_mat)
                new_mat[where_are_NaNs]=1
                new_mat = (new_mat-np.mean(new_mat))/(np.max(new_mat)-np.min(new_mat))        
                testing_dataa.append(new_mat)
                testing_labell.append(k)                                       
test_data = testing_dataa
test_label = testing_labell
print(len(test_data))
print(len(test_label))

In [ ]:
row = len(test_data[0])
column = len(test_data[0][0])
print(row)
print(column)

In [ ]:
test_data_mis = np.array(test_data, 'float32')
test_data_mis = test_data_mis.reshape((len(test_data_mis),row,column,1))

test_label_mis=np.array(test_label)
print(test_data_mis.shape)
print(test_label_mis.shape)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\dis.hdf5'
os.path.exists(filepath)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\raw_1.hdf5'
os.path.exists(filepath)

In [ ]:
#converting the data labels to categorical
test_label_mis = keras.utils.to_categorical(test_label_mis, 7)

print(test_label_mis.shape)

In [ ]:
base_model.load_weights(filepath)
test_pur_5 = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
match_1 = 0
for i in range(len(test_label_mis)):
    if np.argmax(test_label_mis[i]) == np.argmax(test_pur_5[i]):
        match_1+=1
accuracy_1 = (match_1/len(test_label_mis))*100
print('accuracy :', accuracy_1)

# Performing the Model fusion

In [ ]:
dis_6 = test_pur_5
dis_6.shape

In [ ]:
print(dis_1.shape)
print(dis_2.shape)
print(dis_3.shape)
print(dis_4.shape)
print(dis_5.shape)
print(dis_6.shape)

In [ ]:
bal_1 = []
for index,i in enumerate(range(len(test_pur_5))):
#     print(index, np.argmax(test_label_mis[i]), np.argmax(test_pur_5[i]))
    bal_1.append(np.argmax(test_pur_5[i]))

In [ ]:
mon = []
for i in range(len(dis_1)):
    temp = np.vstack((dis_1[i], dis_2[i], dis_3[i],dis_4[i], dis_5[i], dis_6[i],dum[0]))
    mon.append(temp)
mon = np.array(mon)
print(mon.shape)
print(mon.shape[0]*mon.shape[1])

In [ ]:
filepath_new_mis = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\ang_ras_01234101112131420212223.hdf5'
os.path.exists(filepath_new_mis)

In [ ]:
base_model.load_weights(filepath)
test_pur_5 = base_model.predict([test_data_mis],batch_size=32)

In [ ]:
match_1 = 0
for i in range(len(test_label_mis)):
    if np.argmax(test_label_mis[i]) == np.argmax(test_pur_5[i]):
        match_1+=1
accuracy_1 = (match_1/len(test_label_mis))*100
print('accuracy :', accuracy_1)

In [ ]:
os.getcwd()

In [ ]:
m = 0
bal_1 = []
bal_2 = []
# bal_3 = []
for i in range(len(test_label_mis)):
    bal_1.append(test_pur_5[i])
    bal_2.append(np.argmax(test_label_mis[i]))
#     bal_3.append(np.argmax(test_pur_5[i]))

In [ ]:
np.savetxt('Gpr_dis.csv',bal_1, delimiter=',')
np.savetxt('Gr_Tr.csv',bal_2, delimiter=',')
# np.savetxt('bal_3.csv',bal_3, delimiter=',')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_true = []
y_pred = []
for i in range(len(test_label_mis)):
    y = np.argmax(test_label_mis[i])
    y_true.append(y)

for i in range(len(test_pur_5)):
    y = np.argmax(test_pur_5[i]) #Make changes here
    y_pred.append(y)
print(len(y_true))
print(len(y_pred))

In [ ]:

cl_name = np.array(['Action-0','Action-1','Action-2','Action-3','Action-4','Action-5','Action-6'])
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

#     print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cnf_matrix = confusion_matrix(y_true, y_pred)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name,
                      title='Confusion matrix')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# Averaging the model outputs

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Dis_1_01234101112131420212223.hdf5'
base_model.load_weights(filepath)
distance = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Dis_D_1_01234101112131420212223.hdf5'
base_model.load_weights(filepath)
distance_D = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Dis_DD_1_012341011121314202122232425.hdf5'
base_model.load_weights(filepath)
distance_DD = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Ang_1_0123410.hdf5'
base_model.load_weights(filepath)
angle = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Ang_D_1_01234101112131420212223.hdf5'
base_model.load_weights(filepath)
angle_D = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
filepath = 'C:\\Users\\ma2nq\\Desktop\\python\\Kinect_data\\journal\\Ang_DD_1_01.hdf5'
base_model.load_weights(filepath)
angle_DD = base_model.predict([test_data_mis], batch_size=32)

In [ ]:
#Based on individual model
all_list = [distance_n,distance_D_n,distance_DD_n,angle_n,angle_D_n,angle_DD_n]
all_list_name = ['distance','distance_D','distance_DD','angle','angle_D','angle_DD']


for j in range(len(all_list)):
    match_1 = 0
    for i in range(len(test_label_mis)):
        if np.argmax(test_label_mis[i]) == np.argmax(all_list[j][i]):
            match_1+=1
    accuracy_1 = (match_1/len(test_label_mis))*100
    print(all_list_name[j],':',accuracy_1)

In [ ]:
#Based on individual model
all_list = [dis_1,dis_2,dis_3,dis_4,dis_5,dis_6]
all_list_name = ['dis_1','dis_2','dis_3','dis_4','dis_5','dis_6']

for j in range(len(all_list)):
    match_1 = 0
    for i in range(len(test_label_mis)):
        if np.argmax(test_label_mis[i]) == np.argmax(all_list[j][i]):
            match_1+=1
    accuracy_1 = (match_1/len(test_label_mis))*100
    print(all_list_name[j],':',accuracy_1)

In [ ]:
##Prediction ## For 2 sensor FUSION

prediction_max = []
for i in range(len(test_label_mis)):

    x = np.vstack((dis_1[i],dis_2[i],dis_3[i],dis_4[i],dis_5[i],dis_6[i]))

    m = np.max(x, axis=0)
    prediction_max.append(m)
prediction_max = np.array(prediction_max)

prediction_avg = []
for i in range(len(test_label_mis)):

    x = np.vstack((dis_1[i],dis_2[i],dis_3[i],dis_4[i],dis_5[i],dis_6[i]))

    x = np.average(x, axis=0)
    prediction_avg.append(x)
prediction_avg = np.array(prediction_avg)

pred_product = []
for i in range(len(test_label_mis)):
    x = np.vstack((dis_1[i],dis_2[i],dis_3[i],dis_4[i],dis_5[i],dis_6[i]))

    o = np.product(x, axis = 0)
    pred_product.append(o)
 
pred_product = np.array(pred_product)


print(len(prediction_max))
print(len(prediction_avg))
print(len(pred_product))


In [ ]:
#Based on Fusion Method

list_1 = [prediction_max, prediction_avg, pred_product]
list_2 = ['prediction_max','prediction_avg', 'pred_product']

for j in range(len(list_1)):
    correct = 0
    for i in range(len(test_label_mis)):
        if np.argmax(list_1[j][i]) == np.argmax(test_label_mis[i]): #make a change here
            correct+=1
    accuracy = (correct/test_label_mis.shape[0])*100
    print(list_2[j],': {:.2f}'.format(accuracy))

In [ ]:
correct = 0
for i in range(len(test_label_mis)):
    if maj_vote[i] == np.argmax(test_label_mis[i]): #make a change here
        correct += 1
    accuracy = (correct/test_label_mis.shape[0])*100
print('majority voting: {:.2f}'.format(accuracy))        

# code for majority voting

In [ ]:
maj_vote = []
for i in range(len(test_label_mis)):
    v_1 = np.argmax(dis_1[i])
    v_2 = np.argmax(dis_2[i])
    v_3 = np.argmax(dis_3[i])
    v_4 = np.argmax(dis_4[i])
    v_5 = np.argmax(dis_5[i])
    v_6 = np.argmax(dis_6[i])
    lst = [v_1, v_3, v_4, v_5, v_6]

    x_1 = np.vstack((dis_1[i],dis_3[i],dis_4[i],dis_5[i],dis_6[i]))
    x = list(np.average(x_1, axis=0))
    pred = prediction(x,lst)
    maj_vote.append(pred)

In [ ]:
def prediction(avg,lst):
    dic = {}
    dhon = []
    res = list()
    counter = lst
    for i in counter:
        if not i in res:
            res.append(i)
    for i in res:
        m = counter.count(i)
        dic.update({i:m})
    pred_key = list(dic.keys())
    pred_val = list(dic.values())

    val = max(pred_val)    
    thres = pred_val.count(val)
    if thres ==1:
        for output,value in dic.items():
            if value == val:
                return output
    
    if thres ==2:
        for i in dic.items():
            if i[1]==val:
                dhon.append(i[0])          
        can = max(avg[dhon[0]], avg[dhon[1]])
        output = avg.index(can)
        return output    
    
    if thres ==3:
        for i in dic.items():
            if i[1]==val:
                dhon.append(i[0])
        can = max(avg[dhon[0]], avg[dhon[1]], avg[dhon[2]])
        output = avg.index(can)
        return output

In [ ]:
list_1 = [prediction_max, prediction_avg, pred_product]

In [ ]:
for i in range(len(maj_vote)):
    if maj_vote[i] == None:
        print(i,maj_vote[i], True)

In [ ]:
maj_vote= keras.utils.to_categorical(maj_vote, 7)

In [ ]:
y_true = []
y_pred = []
for i in range(len(test_label_mis)):
    y = np.argmax(test_label_mis[i])
    y_true.append(y)

for i in range(len(maj_vote)):
    y = np.argmax(maj_vote[i]) #Make changes here
    y_pred.append(y)
print(len(y_true))
print(len(y_pred))

In [ ]:

cl_name = np.array(['Action-0','Action-1','Action-2','Action-3','Action-4','Action-5','Action-6'])
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')

#     print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cnf_matrix = confusion_matrix(y_true, y_pred)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name,
                      title='Confusion matrix')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cl_name, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
def count_identical_lists(A, B, C, D):
    m = 0
    for i in range(len(list_1)):
        if A[i] == B[i]==C[i] == D[i]:
            m+=1
    return m

In [ ]:
count_identical_lists([0, 0, 0], [0, 0, 0], [2, 1, 0], [2, 1, 0])

In [ ]:
a = [0,1,2,3,4,0]
a.count(0)

In [ ]:
def count_identical_lists(lst1, lst2, lst3, lst4):
    a = [lst1, lst2, lst3, lst4]
    return len([i for i in a if a.count(i) > 1])
    

In [ ]:
def count_identical_lists(lst1, lst2, lst3, lst4):
	lists = [lst1, lst2, lst3, lst4]
	repeat_lists = []
	
	for x in lists:
		count = lists.count(x)
		if count > 1:
			print(x)
			repeat_lists.append(x)
	return len(repeat_lists)

In [ ]:
def count_identical_lists(lst1, lst2, lst3, lst4):
	lst = [lst1, lst2, lst3, lst4]
	for i in lst:
		if lst.count(i) > 1:
			return lst.count(i)
		elif (lst.count(i) == 1 for i in lst):
			return 0

In [ ]:
def is_valid_PIN(pin):
    string = str(pin)
    if len(string)==4 or len(string)==6:
        return('valid')
    else:
        return('not valid')

In [ ]:
is_valid_PIN(1323)

In [ ]:
def is_valid_PIN(pin):
	return len(str(pin)) in [4, 6] and str(pin).isdigit()

In [ ]:
is_valid_PIN(1234)

In [ ]:
def index_of_caps(word):
    return [word.index(i)+1 for i in word if i.isupper()]

In [ ]:
def equal(a, b, c):
    lst = [a,b,c]
    for i in lst:
        if lst.count(i)>1:
            return lst.count(i)
        else:
            return 0

In [ ]:
remove_vowels("I have never seen a thin person drinking Diet Coke.")

In [ ]:
def remove_vowels(txt):
    return ''.join([x for x in txt if x not in 'aeiouAEIOU'])

In [ ]:
remove_dups(["John", "Taylor", "John"]) ➞ ["John", "Taylor"]

In [ ]:
def remove_dups(txt):
    res = []
    for i in txt:
        if i not in res:
            res.append(i) 
    return res

In [ ]:
remove_dups(["John", "Taylor", "John"])

In [ ]:
sum_two_smallest_nums([19, 5, 42, 2, 77]) ➞ 7

In [ ]:
def sum_two_smallest_nums(txt):
    m = sorted(txt)
    return(sum(m[:2]))
    

In [ ]:
def sum_neg(txt):
    return([sum(i) for i in txt if i <= 0])

In [ ]:
sum_neg([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -11, -12, -13, -14, -15])
# There are a total of 10 positive numbers.

In [ ]:
lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -11, -12, -13, -14, -15]
def sum_neg(lst):
    return [sum(1 for x in lst if x>=0), sum(x for x in lst if x<0) ]

In [ ]:
numInStr(["1a", "20", "10", "10"]) ➞ ["1a", "2b"]

In [ ]:
lst = ["20", "10", "10"]
[all(i.isnumeric() for i in lst)]

In [ ]:
return [i for i in lst if any(j.isnumeric() for j in i)]

In [ ]:
def numInStr(lst):
    ress = []
    for i in lst:
        nm = str(i)
        res = []
        for j in range(len(nm)):
            res.append(is_int(nm[j]))
        if True in res:
            ress.append(nm)
    return ress

In [ ]:
'102'.isnumeric()

In [ ]:
ab = 'abc'
def is_int(ab):
    try:
        ab = int(ab)
        return True
    except ValueError:
        return False
    

In [ ]:
def is_int(ab):
    try: 
        int(ab)
        return True
    except ValueError:
        return False

In [ ]:
cars

In [ ]:
dic = {}
res = list()
counter = ['B','B','A','B','C','A','B','B','A','C']
for i in counter:
    if not i in res:
        res.append(i)
for i in res:
    m = counter.count(i)
    dic.update({i:m})
dic

In [ ]:
get_frequencies(["A", "B", "A", "A", "A"]) ➞ { "A" : 4, "B" : 1 }

In [ ]:
def get_frequencies(lst):
    dic = {}
    lst = list(set(lst))
    dic.update({i, lst.count(i)} for i in lst)
    return dic

In [ ]:
get_frequencies(["A", "B", "A", "A", "A"])

In [ ]:
Create a function that takes an array of numbers and return "Boom!"
if the number 7 appears in the array. Otherwise, return "there is no 7 in the array".

In [ ]:
lst = ["A", "B", "A", "107", "A"]
def seven_boom(lst):
    return 'Boom!' if '7' in str(lst) else 'there is no 7 in the list'


In [ ]:
unmix("123456") ➞ "214365"

In [ ]:
a = "hTsii  s aimex dpus rtni.g"
def unmix(a):
    lst = []
    for i in range(0,len(a),2):
        var = a[i:i+2]
        var = var[::-1]
        lst.append(var)
    return ''.join(lst)

In [ ]:
unmix(a)

In [ ]:
arr_1 = ["A", "A", "A"]
arr_2 = ["B"]
# shu_arr= ["A", "B", "A", "B", "A", "B"]

In [ ]:
def bridge_shuffle(lst1, lst2):
    step = 1
    for x in lst2:
        lst1.insert(step,x)
        step += 2
    return lst1

In [ ]:
bridge_shuffle(arr_1, arr_2)

In [ ]:
sampleStr = "Hello!!" 
for elem in sampleStr:
    print(elem)

In [ ]:
Two ice cream stands: A and B each occupy a spot on the beach, from [0, 100]. Their positions are represented 
with coordinates (A, B). One position could be:

In [ ]:
def profit(A, B):
    assert A<B, 'A is greater than B'
    prof_A = 0
    prof_B = 0
    for i in range(0,100+1):
        if abs(A-i)<abs(B-i):
            prof_A+=1
        else :
            prof_B +=1
    return prof_A, prof_B

In [ ]:
first_before_second("a rabbit jumps joyfully", "a", "j") ➞ True
# Every instance of "a" occurs before every instance of "j".

In [ ]:
txt = "a rabbit jumps joyfully"
def first_before_second(txt, a, b):
    lst = []
    for i in txt.split(' '):
        if 'j' in i:
            lst.append('j')
        if 'a' in i:
            lst.append('a')
    a = lst.count('a')
    b = lst.count('j')
    j = 0
    for i in lst:
        if i =='a':
            j+=1
        if i == 'j':
            break
    if a==j:
        return 'True'
    else:
        return 'False'

In [ ]:
def first_before_second(s, first, second):
    return s.rfind(first) < s.find(second)

In [ ]:
word = 'CatBatSatMatGate'
if word.rfind('atee') != -1:
    print(True)
else:
    print(False)


In [ ]:
a = [(2,3),(1,2),(7,3)]
b = [(5,8),(3,4),(5,8),(3,4)]
a = [np.array(i) for i in a]
b = [np.array(i) for i in b]

min([np.linalg.norm(i-j) for i in a for j in b])

In [ ]:
def min_dis(a,b):
    a = [np.array(i) for i in a]
    b = [np.array(i) for i in b]
    minimum = min([np.linalg.norm(i-j) for i in a for j in b])
    return minimum

In [ ]:
a = [(2,3),(1,2)]
b = [(5,8),(3,4),(5,8),(3,4)]

In [ ]:
min_dis(a,b)

In [ ]:
def happy(n):
    while n!=4 and n!=1:
        n = sum([int(i)**2 for i in str(n)])
    return True if n == 1 else False

In [ ]:
flip_list([1, 2, 3, 4]) ➞ [[1], [2], [3], [4]]
# Take a horizontal list and flip it vertical.

flip_list([[5], [6], [9]]) ➞ [5, 6, 9]
# Take a vertical list and flip it horizontal.

flip_list([]) ➞ []

In [ ]:
lst = [1, 2, 3, 4]

In [ ]:
def flip_list(lst):
    if lst==[]:
        return lst
    elif isinstance(lst[0], list):
        return [l[0] for l in lst]
    else:
        return [[l] for l in lst]

flip_list(lst)

In [ ]:
def flip_list(lst):
    if lst == []:
        return lst
    elif isinstance(lst[0], list):
        return [l[0] for l in lst]
    else:
        return [[l]  for l in lst]

In [ ]:
numbers = [1, 2, 3]

result = isinstance(numbers, dict)
print(numbers,'instance of dict?', result)

In [ ]:
get_best_student({
  "John": [100, 90, 80],
  "Bob": [100, 70, 80]
}) ➞ "John"


# Bob's avg = 83.33

In [ ]:
dic = {"John": [100, 90, 80], "Bobbb": [100, 100, 100]}

In [ ]:
import numpy as np
def get_best_student(dic):
    return sorted(dic, key =lambda x:np.average(x[1]), reverse = True)[0][0]

In [ ]:
def get_best_student(dic):
    return sorted(dic, key =lambda x:np.average(x[1]), reverse = True)[0][0]

In [ ]:
get_best_student(dic)

In [ ]:
import collections
# Create a deque
DoubleEnded = collections.deque(["Mon","Tue","Wed"])
print (DoubleEnded)

# Append to the right
print("Adding to the right: ")
DoubleEnded.append("Thu")
print (DoubleEnded)

# append to the left
print("Adding to the left: ")
DoubleEnded.appendleft("Sun")
print (DoubleEnded)

In [ ]:
type(DoubleEnded)

In [ ]:
import cv2
import os
os.getcwd()

In [ ]:
file_path = './s_Tuhin/t_Handl_31/kinect/'
# file_path = './data/s_Zpheng/t_Handl_1/kinect/data.p'
# C:\Users\alami\Desktop\Myo_Paper\DATA_NEW\s_Al-Amin_1
# C:\Users\ma2nq\Desktop\python\Kinect_data\s_Tuhin\t_Handl_31\kinect
# C:\Users\ma2nq\Desktop\python\Kinect_data\s_Tuhin

In [ ]:
img_idx = 0
while True:
    try:        
        img = cv2.imread(file_path + str(img_idx) + '.jpg')
        cv2.putText(img, 'Help', (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        cv2.putText(img, 'f: forward, r: reverse, q: quit', (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        cv2.imshow('img', img)
        k = cv2.waitKey(0)
        if k == ord(' ') or k == ord('f'):
            img_idx += 1
        elif k == ord('r'): 
            img_idx -= 1
        elif k == ord('q'):
            cv2.destroyAllWindows()     
            break        
    except:
        cv2.destroyAllWindows()
        break

In [ ]:
a = np.random.random([1,7])
a

# Code for Majority Voting

In [ ]:
# x_1.shape

In [ ]:
# weight = np.random.random((6,7))
# weight